# AR(1) + GARCH(1,1) Model

## Loading Packages

In [1]:
using Optim

include("jlFiles/printmat.jl")

printlnPs (generic function with 1 method)

In [2]:
using Plots

backend = "gr"              #"gr" (default), "pyplot" 

if backend == "pyplot"
    pyplot(size=(600,400))
    default(show=false)               #for pyplot: avoids pop-ups
else    
    gr(size=(600,400))
    default(show=true)
end

## Loading Data

In [3]:
xx  = readdlm("Data/FFdSizePs.csv",',',header=true)
xx  = xx[1]
ymd = xx[:,1]     #[YearMonthDay, eg. 20121231]
R   = xx[:,2]     #returns for the size portfolio we want to study
xx  = nothing

y = R[2:end]                           #dependent variable, y(t)
x = [ones(size(R,1)-1) R[1:end-1]]     #regressors, [1, y(t-1)]

8324×2 Array{Float64,2}:
 1.0  -1.18
 1.0  -1.37
 1.0   1.73
 1.0   0.77
 1.0   1.41
 1.0   0.68
 1.0   1.26
 1.0   0.61
 1.0   1.11
 1.0   0.63
 1.0   0.64
 1.0   0.37
 1.0   0.49
 ⋮         
 1.0  -0.52
 1.0  -0.3 
 1.0   0.18
 1.0   1.13
 1.0   1.11
 1.0  -0.11
 1.0   0.58
 1.0  -0.53
 1.0  -0.57
 1.0  -0.45
 1.0  -0.35
 1.0  -0.34

## The Likelihood Function

Consider a regression equation, where the residual follows a GARCH(1,1) process

$$
\begin{align*}
y_{t}           &  =x_{t}^{\prime}b+u_{t} \ \text{ with }\ u_{t}=v_{t}\sigma_{t}\\
\sigma_{t}^{2}  &  =\omega+\alpha u_{t-1}^{2}+\beta\sigma_{t-1}^{2}.
\end{align*}
$$
Note that we require $(\omega,\alpha,\beta)$ to all be positive and $\alpha + \beta < 1$.

If $v_{t}\sim N(0,1)$, then the likelihood function is

$$
\ln\mathcal{L}=-\frac{T}{2}\ln\left(  2\pi\right)  -\frac{1}{2}\sum_{t=1}
^{T}\ln\sigma_{t}^{2}-\frac{1}{2}\sum_{t=1}^{T}\frac{u_{t}^{2}}{\sigma_{t}
^{2}}.
$$


The likelihood function of a GARCH(1,1) model can be coded as in
*garch11LL*. The first function calculates time-varying  variances and
the likelihood contributions (for each period). The second functions forms the
loss function used in the minimization.

In [4]:
function garch11LL(parm::Vector,y,x)

  (T,k) = size(x,1,2)

  b     = parm[1:k]         #mean equation, y = x'*b
  (omega,alpha,beta1) = abs.(parm[k+1:k+3])    #GARCH(1,1) equation: 
                                              #s2(t) = omega + alpha*u(t-1)^2 + beta1*s2(t-1)        
  yhat = x*b
  u    = y - yhat
  s2_0 = var(u)                                 #var(u,1) gives a matrix, var(u) a scalar

  s2    = fill(NaN,T)
  s2[1] = omega + alpha*s2_0 + beta1*s2_0        #simple, but slow apparoach
  for t = 2:T                                    #using filter() is perhaps quicker
    s2[t] = omega + alpha*u[t-1]^2 + beta1*s2[t-1]
  end

  LL    = -(1/2)*log(2*pi) - (1/2)*log.(s2) - (1/2)*(u.^2)./s2
  LL[1] = 0               #effectively skip the first observation

  return LL,s2,yhat

end


function garch11LLLoss(parm::Vector,y,x)
    LL,  = garch11LL(parm::Vector,y,x)
    Loss =  -sum(LL)                  #to minimize: -sum(LL)
    return Loss
end


function garch11LLRLoss(parm::Vector,y,x,rho)     #with restriction on alpha+beta1 < 1
  LL, = garch11LL(parm,y,x)
  k = size(x,2)
  (omega,alpha,beta1) = abs.(parm[k+1:k+3])   #s2(t) = omega + alpha*u(t-1)^2 + beta1*s2(t-1)
  g = [alpha + beta1 - 1]                     #alpha + beta1 < 1
  Loss = -sum(LL) + rho*sum(max.(0,g).^2)
  return Loss
end

garch11LLRLoss (generic function with 1 method)

## Maximize the Likelihood Function

In [5]:
#mean equation, y = x'b
#GARCH(1,1) equation: s2(t) = omega + alfa*u(t-1)^2 + beta1*s2(t-1)

par0 = [mean(y);0;var(y)*0.05;0.05;0.9]         #initial parameter guess

(loglik,s2,yhat) = garch11LL(par0,y,x)            #just testing the log lik
LL               = garch11LLLoss(par0,y,x)
printlnPs("Value of (-)log-likelihood fn at starting guess: ",LL)

Sol = optimize(par->garch11LLLoss(par,y,x),par0)   #do MLE by optimization with optimize, minimize -sum(LL)

parHat            = Optim.minimizer(Sol)          #extract the optimal solution 
parHat[end-2:end] = abs.(parHat[end-2:end])       #since the likelihood function uses abs(these values)

LLHat             = garch11LLLoss(parHat,y,x)
printlnPs("Value of (-)log-likelihood fn at estimate: ",LLHat)

println("\nParameter estimates (b[1],b[2],omega,alpha,beta1): ")
printmat(parHat)

Value of (-)log-likelihood fn at starting guess:   9231.913
Value of (-)log-likelihood fn at estimate:   8243.608

Parameter estimates (b[1],b[2],omega,alpha,beta1): 
     0.049
     0.256
     0.014
     0.162
     0.824



In [6]:
println("\nIterate with harder and harder punishment on α + β < 1 restriction. PERHAPS NOT NEEDED")

options = Optim.Options(show_trace=false,show_every=10)
for rho = 0.0:0.1:1
  Sol = optimize(par->garch11LLRLoss(par,y,x,rho),par0,options)
  par = Optim.minimizer(Sol)
  par[end-3:end-1] = abs.(par[end-3:end-1])
  printmat([rho par'])
  par0 = deepcopy(par)
end


Iterate with harder and harder punishment on α + β < 1 restriction. PERHAPS NOT NEEDED
     0.000     0.049     0.256     0.014     0.162     0.824

     0.100     0.049     0.256     0.014     0.162     0.824

     0.200     0.049     0.256     0.014     0.162     0.824

     0.300     0.049     0.256     0.014     0.162     0.824

     0.400     0.049     0.256     0.014     0.162     0.824

     0.500     0.049     0.256     0.014     0.162     0.824

     0.600     0.049     0.256     0.014     0.162     0.824

     0.700     0.049     0.256     0.014     0.162     0.824

     0.800     0.049     0.256     0.014     0.162     0.824

     0.900     0.049     0.256     0.014     0.162     0.824

     1.000     0.049     0.256     0.014     0.162     0.824



## Value at Risk

In [7]:
(_,s2,ER) = garch11LL(parHat,y,x)
VaR95     = -(ER - 1.64*sqrt.(s2))

CovRatio = mean((-y) .>= VaR95)             #coverage ratio for VaR
printlnPs("\nCoverage ratio for VaR(95%): ",CovRatio)


Coverage ratio for VaR(95%):      0.059


In [8]:
y   = floor.(ymd/10000)                        #converting 20121231 to Date
m   = floor.( (ymd-y*10000)/100 )
d   = ymd - y*10000 - m*100

dN = Array{Date}(length(ymd))
for t = 1:length(ymd)
    dN[t] = Date(y[t],m[t],d[t])
end      


xTicks = Integer.([Date(1990);Date(2000);Date(2010)])     #controlling the tick locations
plot(dN[2:end],VaR95,xticks=xTicks,legend=false)
title!("1-day VaR (95%)")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1990-01-01 
 
 
 2000-01-01 
 
 
 2010-01-01 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 8 
 
 
 10 
 
 
 1-day VaR (95%) 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 21.7148,333.969 21.7624,329.968 21.9054,344.629 21.9531,341.51 22.0007,347.009 22.0484,345.536 22.0961,350.878 22.239,349.696 22.2867,354.727 22.3344,354.424 
 22.382,357.71 22.4297,358.983 22.5727,362.87 22.6203,366.158 22.668,349.599 22.7156,361.732 22.7633,364.086 22.9063,365.617 22.9539,368.198 23.0016,356.808 
 23.0493,365.511 23.0969,363.107 23.2399,367.462 23.2876,367.643 23.3352,370.984 23.3829,374.496 23.4305,375.829 23.5735,375.902 23.6212,363.473 23.6688,369.99 
 23.7165,372.581 23.7642,354.586 23.9548,358.864 24.0025,360.158 24.0501,369.651 24.0978,360.623 24.2408,361.082 24.2884,353.525 24.3361,365.791 24.3837,364.7 
 24.4314,368.762 24.5744,372.787 24.622,361.776 24.6697,356.526 24.7174,343.179 24.765,304.169 24.908,316.829 24.9557,306.172 25.0033,332.789 25.051,333.799 
 25.0986,337.763 25.2416,342.803 25.2893,291.485 25.3369,313.349 25.3846,333.826 25.4322,327.673 25.5752,334.005 25.6229,285.432 25.6705,293.437 25.7182,316.159 
 25.7659,165.735 25.9088,230.888 25.9565,225.083 26.0042,237.759 26.0518,254.914 26.2425,258.845 26.2901,248.777 26.3378,277.906 26.3854,294.433 26.4331,304.679 
 26.5761,309.786 26.6237,305.584 26.6714,317.097 26.7191,319.666 26.7667,321.257 26.9097,335.722 26.9574,326.496 27.005,339.662 27.0527,335.698 27.1003,341.017 
 27.2433,333.588 27.291,344.546 27.3386,349.998 27.3863,353.117 27.434,351.845 27.5769,359.016 27.6246,362.981 27.6723,366.219 27.7199,369.48 27.7676,363.376 
 27.9106,365.311 27.9582,368.556 28.0059,373.605 28.0535,375.004 28.1012,374.76 28.2442,375.661 28.2918,376.652 28.3395,376.826 28.3872,377.199 28.4348,379.722 
 28.6255,378.375 28.6731,374.621 28.7208,375.641 28.7684,367.681 28.9114,374.222 28.9591,373.413 29.0067,374.455 29.0544,378.083 29.1021,375.956 29.245,377.256 
 29.2927,377.117 29.3404,378.541 29.388,380.483 29.4357,374.394 29.5787,378.946 29.6263,376.841 29.674,378.317 29.7216,378.424 29.7693,375.474 29.9123,378.663 
 29.9599,381.592 30.0076,382.016 30.0553,381.656 30.1029,376.548 30.2459,379.431 30.2935,364.374 30.3412,372.964 30.3889,376.496 30.5795,376.918 30.6272,374.132 
 30.6748,372.129 30.7225,376.431 30.7701,372.382 30.9131,376.895 30.9608,375.708 31.0084,370.297 31.0561,373.209 31.1038,374.147 31.2467,372.803 31.2944,372.705 
 31.3421,366.283 31.3897,371.465 31.4374,374.983 31.5804,372.723 31.628,376.506 31.6757,378.797 31.7233,378.032 31.771,368.789 31.914,374.31 31.9616,370.415 
 32.0093,374.796 32.057,377.107 32.1046,376.845 32.2476,374.072 32.2953,376.01 32.3429,365.306 32.3906,372.24 32.4382,372.408 32.5812,369.165 32.6289,345.886 
 32.6765,355.608 32.7242,364.567 32.7719,365.037 32.9148,362.526 32.9625,360.046 33.0102,364.537 33.0578,363.832 33.1055,360.361 33.2961,368.794 33.3438,370.268 
 33.3914,370.171 33.4391,363.337 33.5821,368.112 33.6297,361.085 33.6774,368.835 33.7251,371.514 33.7727,370.575 33.9157,370.077 33.9634,369.417 34.011,371.977 
 34.0587,371.31 34.1063,363.021 34.2493,368.403 34.297,363.417 34.3446,349.442 34.3923,360.987 34.44,361.46 34.5829,330.406 34.6306,301.06 34.6783,330.549 
 34.7259,331.036 34.7736,337.83 34.9165,344.326 34.9642,346.69 35.0119,342.882 35.0595,351.337 35.1072,351.286 35.2502,356.124 35.2978,361.29 35.3455,358.871 
 35.3931,363.931 35.4408,350.394 35.5838,360.182 35.6314,364.773 35.6791,362.258 35.7268,369.68 35.7744,351.451 35.9174,363.434 35.9651,351.718 36.0127,357.49 
 36.0604,365.641 36.108,356.454 36.251,365.549 36.3463,369.107 36.394,371.182 36.4417,342.364 36.5846,353.928 36.6323,358.582 36.68,365.652 36.7276,366.036 
 36.7753,364.598 36.9183,364.113 36.9659,360.73 37.0136,366.646 37.0612,367.161 37.1089,370.237 37.2519,362.496 37.2995,359.123 37.3472,368.46